# DAMO-630-29 Assignment 01

In [60]:
# pip install scikit-learn pandas numpy matplotlib seaborn sdv 
#attention 

In [61]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sdv.metadata import Metadata
from sdv.single_table import GaussianCopulaSynthesizer, CTGANSynthesizer
from sdmetrics.reports.single_table import QualityReport, DiagnosticReport

## BUSINESS CHALLENGE #01

# TASK I - Exploratory Data Analysis

The EDA offers an initial overview of the dataset by inspecting its structure, detecting missing values or outliers, and applying descriptive statistics with visualizations. These insights provide the foundation for subsequent synthetic data generation and evaluation.

In [ ]:
# Cell C — Recreate `df_for_clustering` & `scaler_model` if missing (idempotent)
from pyspark.sql import DataFrame as _SparkDF
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler, StandardScaler

df = globals().get('df', None)
if globals().get('df_for_clustering', None) is not None:
    print('✅ `df_for_clustering` already present — no action taken.')
else:
    if df is None:
        print('⚠️ `df` not found. Please run BC2.I cell or run Cell B to reload the dataset first.')
    else:
        try:
            import pandas as _pd
        except Exception:
            _pd = None

        if isinstance(df, _SparkDF):
            candidates = ['trip_distance','fare_amount','tip_amount','tolls_amount','total_amount']
            present = globals().get('clustering_numeric_cols', None)
            if not present:
                present = [c for c in candidates if c in df.columns]
                globals()['clustering_numeric_cols'] = present
            if not present:
                raise RuntimeError('No numeric columns available for clustering.')

            # Build cleaned DataFrame similar to Step 2
            dfk = df.select(*[F.col(c).cast('double').alias(c) for c in present])
            dfk = dfk.na.drop(subset=present)
            for c in present:
                if c in ('trip_distance', 'fare_amount', 'total_amount'):
                    dfk = dfk.filter(F.col(c) > 0)
                else:
                    dfk = dfk.filter(F.col(c) >= 0)

            # Optional: trim extreme outliers (same logic as Step 2)
            trim_bounds = {}
            for c in present:
                try:
                    p999 = dfk.approxQuantile(c, [0.999], 0.01)[0]
                    if p999 is not None:
                        trim_bounds[c] = p999
                        dfk = dfk.filter(F.col(c) <= F.lit(p999))
                except Exception:
                    pass

            print('✅ Cleaning done. Rows after cleaning:', dfk.count())
            if trim_bounds:
                print('Applied p99.9 trims:', trim_bounds)

            # Assemble & scale
            assembler = VectorAssembler(inputCols=present, outputCol='features')
            vec = assembler.transform(dfk)
            scaler = StandardScaler(inputCol='features', outputCol='features_scaled', withMean=True, withStd=True)
            scaler_model = scaler.fit(vec)
            vecs = scaler_model.transform(vec).select(*present, 'features_scaled').cache()

            print('✅ Vectorization & scaling complete. Sample:')
            vecs.show(5, truncate=False)

            globals()['df_for_clustering'] = vecs
            globals()['scaler_model'] = scaler_model
            print('✅ Stored `df_for_clustering` and `scaler_model` in globals.')

        else:
            # pandas path: try to convert to Spark if a Spark session exists, otherwise notify user
            if _pd is not None and isinstance(df, _pd.DataFrame):
                spark = globals().get('spark', None)
                if spark is None:
                    print('⚠️ `df` is a pandas DataFrame and no Spark session is available.

































                print('⚠️ Unsupported `df` type. Expected Spark or pandas DataFrame.')            else:                        print('❌ Conversion to Spark failed or subsequent processing failed:', e_conv)                    except Exception as e_conv:                        print('✅ Created `df_for_clustering` after converting pandas to Spark.')                        globals()['scaler_model'] = scaler_model                        globals()['df_for_clustering'] = vecs                        vecs = scaler_model.transform(vec).select(*present, 'features_scaled').cache()                        scaler_model = scaler.fit(vec)                        scaler = StandardScaler(inputCol='features', outputCol='features_scaled', withMean=True, withStd=True)                        vec = assembler.transform(dfk)                        assembler = VectorAssembler(inputCols=present, outputCol='features')                                dfk = dfk.filter(F.col(c) >= 0)                            else:                                dfk = dfk.filter(F.col(c) > 0)                            if c in ('trip_distance', 'fare_amount', 'total_amount'):                        for c in present:                        dfk = dfk.na.drop(subset=present)                        dfk = sdf.select(*[F.col(c).cast('double').alias(c) for c in present])                            raise RuntimeError('No numeric columns available for clustering after conversion.')                        if not present:                            globals()['clustering_numeric_cols'] = present                            present = [c for c in candidates if c in sdf.columns]                        if not present:                        present = globals().get('clustering_numeric_cols', None)                        candidates = ['trip_distance','fare_amount','tip_amount','tolls_amount','total_amount']                        # Proceed with the same Spark path after conversion                        print('✅ Conversion to Spark DataFrame succeeded — re-running Spark path.')                        globals()['df'] = sdf                        sdf = spark.createDataFrame(df)                    try:                    print('ℹ️ Converting pandas DataFrame to Spark DataFrame then proceeding (this may be slow).')                else:KMeans steps require Spark (pyspark.ml). Either start Spark or convert the workflow to scikit-learn.')